# Examen Final: Clasificación de Mortalidad Hospitalaria  
**Machine Learning Supervisado**  

# Mateo Hernández Gualdron


*September 25, 2025*

---

## Propósito  
El propósito de este examen es evaluar el cumplimiento de los objetivos de aprendizaje planteados al comienzo del curso:

- Identificar problemas que se pueden resolver usando Machine Learning supervisado.  
- Implementar una solución de Machine Learning supervisado a problemas prácticos.  
- Evaluar el desempeño de modelos de Machine Learning supervisado.  

---

## 1. Descripción del Problema  
Predecir la mortalidad hospitalaria de pacientes críticamente enfermos es importante debido a la creciente preocupación sobre la pérdida de control de los pacientes hacia el final de la vida.  

Una predicción acertada permite tomar decisiones anticipadas para reducir la frecuencia de un proceso de muerte mecánico, doloroso y prolongado.  

El objetivo del examen es diseñar un clasificador que permita predecir la mortalidad de estos pacientes a partir de varias características fisiológicas, demográficas y de severidad de la enfermedad.  

---

## 2. Dataset  
El archivo de datos proporcionado es **`data_train.csv`**. Este archivo contiene un total de **9105 registros de pacientes**.  

Las características del dataset incluyen:  
- Información fisiológica de los pacientes.  
- Datos demográficos.  
- Información sobre la severidad de la enfermedad.  
- Indicadores de mortalidad a 2 y 6 meses.  
- Columna de mortalidad hospitalaria (`hospdead`, la etiqueta a predecir).  

---

## 3. Evaluación  
Usted es libre de emplear cualquier tipo de modelo, así como de utilizar el preprocesamiento de los datos que considere adecuado.  

La evaluación del examen final se basará en los siguientes criterios:

- **(10%)** Adecuación y preprocesamiento de los datos.  
- **(10%)** Evidencia del entrenamiento apropiado de sus modelos.  
- **(20%)** Selección del tipo de modelo, y método de selección de modelo y/o regularización utilizados, incluyendo evidencia numérica.  
- **(20%)** Calidad de su modelo final. Se debe especificar y justificar claramente el criterio de desempeño utilizado.  
  - ¿Cuál es el desempeño esperado de su modelo en datos futuros?  
  - Dé evidencia numérica.  
- **(20%)** Análisis de sus resultados.  
- **(20%)** Calidad y orden del informe.  

---

## 4. Entregables  
El entregable es un informe (**Jupyter Notebook**) del procedimiento llevado a cabo para llegar a su modelo final.  

Este informe debe estar bien estructurado e incluir la información requerida en la evaluación. Incluya **gráficas y tablas** que le permitan presentar la información de manera concisa y clara.  

El código debe estar bien estructurado y apropiadamente comentado. En el notebook deben visualizarse las ejecuciones realizadas.  


In [2]:
import pandas as pd
data_raw = pd.read_csv('data_train.csv',sep=',')
data_raw.drop('Unnamed: 0', axis=1, inplace=True)
data_raw.head(6)

,age,sex,dzgroup,dzclass,num.co,scoma,avtisst,race,sps,aps,...,dnr,dnrday,meanbp,hrt,resp,temp,crea,sod,adlsc,hospdead
0,62.84998,male,Lung Cancer,Cancer,0,0.0,7.000000,other,33.898438,20.0,...,no dnr,5.0,97.0,69.0,22.0,36.00000,1.199951,141.0,7.0,0
1,60.33899,female,Cirrhosis,COPD/CHF/Cirrhosis,2,44.0,29.000000,white,52.695312,74.0,...,NaN,NaN,43.0,112.0,34.0,34.59375,5.500000,132.0,1.0,1
2,52.74698,female,Cirrhosis,COPD/CHF/Cirrhosis,2,0.0,13.000000,white,20.500000,45.0,...,no dnr,17.0,70.0,88.0,28.0,37.39844,2.000000,134.0,0.0,0
3,42.38498,female,Lung Cancer,Cancer,2,0.0,7.000000,white,20.097656,19.0,...,no dnr,3.0,75.0,88.0,32.0,35.00000,0.799927,139.0,0.0,0
4,79.88495,female,ARF/MOSF w/Sepsis,ARF/MOSF,1,26.0,18.666656,white,23.500000,30.0,...,no dnr,16.0,59.0,112.0,20.0,37.89844,0.799927,143.0,2.0,0
5,93.01599,male,Coma,Coma,1,55.0,5.000000,white,19.398438,27.0,...,no dnr,4.0,110.0,101.0,44.0,38.39844,0.699951,140.0,1.0,1


In [6]:
# nulos y duplicados
print(f'Datos nulos: {data_raw.isnull().sum().sum()}')
print(f'Datos duplicados: {data_raw.duplicated().sum().sum()}')

Datos nulos: 261
Datos duplicados: 0


In [8]:
# Exploración de datos nulos por columna
data_raw.isnull().sum()

age          0
sex          0
dzgroup      0
dzclass      0
num.co       0
scoma        1
avtisst     82
race        42
sps          1
aps          1
surv2m       1
surv6m       1
hday         0
diabetes     0
dementia     0
ca           0
dnr         30
dnrday      30
meanbp       1
hrt          1
resp         1
temp         1
crea        67
sod          1
adlsc        0
hospdead     0
dtype: int64